# Bidding

This is a tutorial for how to use the bidding engine

In [1]:
import sys
sys.path.append('../../src')

from nn.models import Models
from bots import BotBid
from util import hand_to_str
from sample import Sample
import conf
import numpy as np

np.set_printoptions(precision=2, suppress=True, linewidth=200)
np.random.seed(42)


Instructions for updating:
non-resource variables are not supported in the long term


In [2]:
models = Models.from_conf(conf.load('../Conf/UCBC2024.conf'),'..')   # loading neural networks
sampler = Sample.from_conf(conf.load('../Conf/UCBC2024.conf'), False)  # Load sampling strategies


INFO:tensorflow:Restoring parameters from ..\Models/NS1EW99-bidding_same-5556000
INFO:tensorflow:Restoring parameters from ..\Models/NS1EW99-binfo_same-5556000
INFO:tensorflow:Restoring parameters from ..\Models/lead_suit-999000
INFO:tensorflow:Restoring parameters from ..\Models/lead_nt-475000
INFO:tensorflow:Restoring parameters from ..\Models/Jack/lr3-1000000
INFO:tensorflow:Restoring parameters from ..\Models/single_dummy-32768000
INFO:tensorflow:Restoring parameters from ..\Models/lefty_nt-475000
INFO:tensorflow:Restoring parameters from ..\Models/dummy_nt-475000
INFO:tensorflow:Restoring parameters from ..\Models/righty_nt-475000
INFO:tensorflow:Restoring parameters from ..\Models/decl_nt-950000
INFO:tensorflow:Restoring parameters from ..\Models/lefty_suit-999000
INFO:tensorflow:Restoring parameters from ..\Models/dummy_suit-999000
INFO:tensorflow:Restoring parameters from ..\Models/righty_suit-999000
INFO:tensorflow:Restoring parameters from ..\Models/decl_suit-1278000


### Running through an example

In [3]:
# East deals, EW vulnerable.
vuln_ns, vuln_ew = False, False

# you sit West and hold:
hand = 'AQ8.KQ94.JT.KJ52'

auction = ["PAD_START", "PAD_START", "PAD_START", "PASS"]
bot_bid = BotBid([vuln_ns, vuln_ew], hand, models, sampler, 0, 0, False)

bid = bot_bid.bid(auction)
bid.to_dict()['candidates']

# what's your bid?

[{'call': '1N', 'insta_score': 0.967}]

In [4]:
auction = ["PAD_START", "PAD_START", "PAD_START", "PASS", "1N", "PASS", "3S","PASS"]
bid = bot_bid.bid(auction)

bid.to_dict()['candidates']

[{'call': '4H', 'insta_score': 0.531, 'expected_score': 492, 'adjustment': 27},
 {'call': '3N', 'insta_score': 0.3, 'expected_score': 415, 'adjustment': 15},
 {'call': '4C', 'insta_score': 0.09, 'expected_score': 316, 'adjustment': 5}]

in the output above:
- `insta_score` reflects what the neural network prefers to bid
- `expected_score` is based on what you expect to score on difference samples (for example, 70% of the time you are making and 30% of the time you are going one off, then the expected score is 0.7 * 620 - 0.3 * 100)

### Samples consistent with the auction so far

Above we said that we are computig expected scores on samples consistent with the auction so far.

How do we get these samples?

We get some samples in the response we get from the bot (i.e `bid.samples`)

In [5]:
bid.samples

['AQ8.KQ9x.JT.KJxx KTxxxx.xxx.K9x.x .AJTx.A8xxx.A98x J9xx.8x.Qxx.QTxx 0.95894 \n P-1N-P-3S-P-4H-P-4N-P-5H-P-6H-P-P-P (12.0, 12.3)  \n P-1N-P-3S-P-3N-P-P-P (10.9, 11.3)  \n P-1N-P-3S-P-4C-P-4N-P-5H-P-6C-P-P-P (12.1, 12.4) ',
 'AQ8.KQ9x.JT.KJxx KJxx.Jxx.Axxx.8x .Axxx.KQ9xx.AT9x T9xxxx.T8.8x.Qxx 0.95832 \n P-1N-P-3S-P-4H-P-4N-P-5H-P-6H-P-P-P (12.0, 12.1)  \n P-1N-P-3S-P-3N-P-P-P (9.9, 11.4)  \n P-1N-P-3S-P-4C-P-4N-P-5H-P-6C-P-P-P (12.4, 12.1) ',
 'AQ8.KQ9x.JT.KJxx KJTxx.A8x.98.xxx .Jxxx.AKQxx.QT8x 9xxxx.Tx.xxxx.A9 0.94991 \n P-1N-P-3S-P-4H-P-P-P (10.9, 11.4)  \n P-1N-P-3S-P-3N-P-P-P (8.4, 10.0)  \n P-1N-P-3S-P-4C-P-4N-P-5H-P-6C-P-P-P (11.0, 11.2) ',
 'AQ8.KQ9x.JT.KJxx Kxxxx.xxx.9x.Txx .AT8x.KQ8xx.AQ9x JT9xx.Jx.Axxx.8x 0.94387 \n P-1N-P-3S-P-4H-P-4N-P-5H-P-6H-P-P-P (12.3, 12.3)  \n P-1N-P-3S-P-3N-P-4C-P-4S-P-5C-P-5H-P-6N-P-P-P (10.8, 11.8)  \n P-1N-P-3S-P-4C-P-4N-P-5H-P-6C-P-P-P (12.4, 12.2) ',
 'AQ8.KQ9x.JT.KJxx KJxx.T8xx.Kxxx.x .AJxx.A98xx.A98x T9xxxx.x.Qx.QTxx 0.94351 \n P-1N-P-3S-P-4H-

each row is one sample. the hands are in the order: N E S W

(West always has the same cards because it's your hand and it's known to you)

In [6]:
# we got some samples above. if we want more, we can always get more

sample_hands, sorted_score, p_hcp, p_shp, good_quality = bot_bid.sample_hands_for_auction(["PAD_START", "PAD_START", "PAD_START", "PASS", "1N", "PASS", "3S","PASS"],0)
print("Good quality:", good_quality)
for i in range(sample_hands.shape[0]):
    print(f'{hand_to_str(sample_hands[i, 0, :])} {hand_to_str(sample_hands[i, 1, :])} {hand_to_str(sample_hands[i, 2, :])} {hand_to_str(sample_hands[i, 3, :])}')

Good quality: True
AQ8.KQ9x.JT.KJxx KTxxxx.xxx.Kx.Qx .AT8x.Qxxxx.AT98 J9xx.Jx.A98x.xxx
AQ8.KQ9x.JT.KJxx JTxxx.Txx.Axx.9x .A8xx.KQ9xx.Axxx K9xxx.Jx.8xx.QT8
AQ8.KQ9x.JT.KJxx J9xxxx.Tx.A8x.Qx .A8xx.KQ9xx.AT9x KTxx.Jxx.xxx.8xx
AQ8.KQ9x.JT.KJxx J9xxxx.x.Axx.QT9 .Axxx.KQ98x.A8xx KTxx.JT8x.xxx.xx
AQ8.KQ9x.JT.KJxx JT9xxx.xx.xxx.xx .AJTx.AQ8xx.AQT9 Kxxx.8xx.K9x.8xx
AQ8.KQ9x.JT.KJxx KJ9xx.Txx.Qxx.xx .A8xx.K9xxx.A98x Txxxx.Jx.A8x.QTx
AQ8.KQ9x.JT.KJxx KT9xx.8x.Q9xx.8x .ATxx.Axxx.AT9xx Jxxxx.Jxx.K8x.Qx
AQ8.KQ9x.JT.KJxx KJTxxx.xxx.8x.9x .A8xx.Axxx.A8xxx 9xxx.JT.KQ9xx.QT
AQ8.KQ9x.JT.KJxx KTxxxx.Jxx.8x.xx .Axxx.Qxxx.AQ8xx J9xx.T8.AK9xx.T9
AQ8.KQ9x.JT.KJxx Jxxxx.JT8.8xx.xx T.Axxx.AKQx.T98x K9xx.xx.9xxx.AQx
AQ8.KQ9x.JT.KJxx KJ9xx.xx.Qxx.T8x .Axxx.A9xx.A9xxx Txxxx.JT8.K8xx.Q
AQ8.KQ9x.JT.KJxx KJxx.Jx.Axxx.T9x .Axxx.Q98x.AQ8xx T9xxxx.T8x.Kxx.x
AQ8.KQ9x.JT.KJxx K9xxxx.J.9xxx.T9 .T8xx.KQxx.AQxxx JTxx.Axxx.A8x.8x
AQ8.KQ9x.JT.KJxx JTxxx.Tx.Q9xx.Ax .A8xx.AK8x.Q98xx K9xxx.Jxx.xxx.Tx
AQ8.KQ9x.JT.KJxx KJxx.Jxx.xxx